# Test Mercado libre api
https://developers.mercadolibre.com.co/es_ar/mi-primera-aplicacion

https://developers.mercadolibre.com.co/devcenter

https://www.youtube.com/watch?v=lvPAGzUFacE&ab_channel=miguelsanchezco

In [1]:
# Definiciones api Mercadolibre
appId='8632998328956624'
secret = 'IrkatetIZeHYUkRcr95xPhcOPbLoW7zA'
redirect_url = 'https://app-g57fixcgxs9zbvvwhqqqqz.streamlit.app'
url_auth = 'https://auth.mercadolibre.com.co/authorization?response_type:code&client_id:8632998328956624&redirect_uri:https://app-g57fixcgxs9zbvvwhqqqqz.streamlit.app'
# Cada vez que se necesite un nuevo access_token se debe hacer un POST a nuevamente a la url auth
code='TG-664a905023da560001cb90e9-136187407'

# Resultados del tocken
access_token = "APP_USR-8632998328956624-051919-6dd74dd53fc22c7a0672e68ea56b18dc-136187407"
refresh_token = "TG-664a9081e7be1f000157fdfe-136187407"
user_id = "136187407"
headers = {'Authorization': f'Bearer {access_token}'}

site_id = 'MCO'


In [27]:
# Cambiar code por el token de autorización / 6 horas con toker refresh toek
# Run only the fist time to get the access token
import requests
import json

url = 'https://api.mercadolibre.com/oauth/token'
headers = {'accept': 'application/json', 'content-type': 'application/x-www-form-urlencoded'}
data = {
    'grant_type': 'authorization_code',
    'client_id': appId,
    'client_secret': secret,
    'code': code,
    'redirect_uri': redirect_url,
    'code_verifier': ''
}

response = requests.post(url, headers=headers, data=data)
print(json.dumps(response.json(), indent=4))


{
    "access_token": "APP_USR-8632998328956624-051919-6dd74dd53fc22c7a0672e68ea56b18dc-136187407",
    "token_type": "Bearer",
    "expires_in": 21600,
    "scope": "offline_access read write",
    "user_id": 136187407,
    "refresh_token": "TG-664a9081e7be1f000157fdfe-136187407"
}


In [ ]:
# Validar usuario
import requests
headers = {'Authorization': f'Bearer {access_token}'}
url = f'https://api.mercadopago.com/users/{user_id}'
response = requests.get(url, headers=headers)
print(json.dumps(response.json(), indent=4))

In [ ]:
response.json()['available_filters']
# Filto por año
#'id': 'VEHICLE_YEAR', 'values': [{'id': '[2025-2025]'},
# Filtro por condición
#{'id': 'ITEM_CONDITION',  'values': [{'id': '2230581', 'name': 'Usado', 'results': 60107}

In [5]:
import requests

def standarize_str(s:str) ->str:
    return s.lower().replace(' ', '_').replace('(', '').replace(')', '').replace('.','').replace('/','_').replace('-','_').replace('__','_')\
            .replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

# Definiciones script

# Years to search until 2024
year_min = 2010
year_max = 2024
max_offset = 4000 # for public users
results = 50 # 50 items per call
ciclos = max_offset // results
print(f'ciclos: {ciclos}')

key_word = 'Carros' #VEHICLE_YEAR
category_id = 'MCO1744' 
year_list = [year for year in range(year_min, year_max+1)]
print(year_list) #[2025-2025]



ciclos: 80
[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [7]:
# lista items categorias autos, motos y otros MCO1744 

total_results = []
for year in year_list:
    for offset in range(0, max_offset, results):


        url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={category_id}&ITEM_CONDITION=2230581&VEHICLE_YEAR=[{year}-{year}]&offset={offset}' 
        response = requests.get(url, headers=headers)
        num_results = len(response.json()['results'])
        print(f'=== Year:{year} - {response.json()["paging"]["total"]} - Results:{num_results} - offset:{offset} ===')    
        if num_results == 0:
            break
        else:
            for i in range(0, len(response.json()['results'])):
                try:
                    result = {
                        'id_car' :  response.json()['results'][i]['id'] ,
                        'title': response.json()['results'][i]['title'],
                        'condition' : response.json()['results'][i]['condition'],
                        'link' : response.json()['results'][i]['permalink'],
                        'category_id' : response.json()['results'][i]['category_id'],
                        'catalog_product_id' : response.json()['results'][i]['catalog_product_id'],
                        'domain_id' : response.json()['results'][i]['domain_id'],
                        'currency_id' : response.json()['results'][i]['currency_id'],
                        'price' : response.json()['results'][i]['price'],
                        'seller_id' : response.json()['results'][i]['seller']['id'],
                        'seller_name' : response.json()['results'][i]['seller']['nickname'],
                        'country' : response.json()['results'][i]['location']['country']['name'],
                        'city' : response.json()['results'][i]['location']['state']['name'],
                        'stop_time' : response.json()['results'][i]['stop_time'],
                    }
                    attributes = response.json()['results'][i]['attributes']
                    for a in attributes:
                        result[standarize_str(a['name'])] = a['value_name']
                    #print(f'\t{i} - {result["title"]} - año:{result["año"]} - attr:{len(attributes)}')
                    
                    # Se agrega al resultado total
                    total_results.append(result)
                except:
                    print(f'Error en el item {i}')
                    pass

print(f'Resultados totales = {len(total_results)}')


=== Year:2010 - 1047 - Results:50 - offset:0 ===
=== Year:2010 - 1047 - Results:50 - offset:50 ===
=== Year:2010 - 1047 - Results:50 - offset:100 ===
=== Year:2010 - 1047 - Results:50 - offset:150 ===
=== Year:2010 - 1047 - Results:50 - offset:200 ===
=== Year:2010 - 1047 - Results:50 - offset:250 ===
=== Year:2010 - 1047 - Results:50 - offset:300 ===
=== Year:2010 - 1047 - Results:50 - offset:350 ===
=== Year:2010 - 1047 - Results:50 - offset:400 ===
=== Year:2010 - 1047 - Results:50 - offset:450 ===
=== Year:2010 - 1047 - Results:50 - offset:500 ===
=== Year:2010 - 1047 - Results:50 - offset:550 ===
=== Year:2010 - 1047 - Results:50 - offset:600 ===
=== Year:2010 - 1047 - Results:50 - offset:650 ===
=== Year:2010 - 1047 - Results:50 - offset:700 ===
=== Year:2010 - 1047 - Results:50 - offset:750 ===
=== Year:2010 - 1047 - Results:50 - offset:800 ===
=== Year:2010 - 1047 - Results:50 - offset:850 ===
=== Year:2010 - 1047 - Results:50 - offset:900 ===
=== Year:2010 - 1047 - Results:50 

In [8]:
len(total_results)

43348

In [9]:
# save results in a json file
import json
json_file = 'total_car_list.json'
# Save data to json
with open(f'..\data\car_attributes.json', 'w') as file:
    json.dump(total_results, file)


In [147]:
import pandas as pd
df = pd.read_json('..\data\car_attributes.json')
print(f'df {df.shape}')
df.head()

df (8819, 43)


,id_car,title,condition,link,category_id,catalog_product_id,domain_id,currency_id,price,seller_id,...,aceleracion_nafta,autonomia_con_nafta,consumo_promedio_nafta,cilindros,modelo_detallado,marca_fipe,modelo_fipe,año_fipe,velocidad_maxima_con_nafta,transmisi��n
0,MCO2391168700,Kia Niro Emotion 1.6,used,https://carro.mercadolibre.com.co/MCO-23911687...,MCO1744,MCO21073827,MCO-CARS_AND_VANS,COP,130000000,1820113826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCO2391201640,Suzuki Swift 1.2,used,https://carro.mercadolibre.com.co/MCO-23912016...,MCO1744,MCO21073612,MCO-CARS_AND_VANS,COP,67000000,1819957298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCO2391198152,Mazda Cx30 Grand Touring 2.0,used,https://carro.mercadolibre.com.co/MCO-23911981...,MCO1744,MCO27942130,MCO-CARS_AND_VANS,COP,102900000,1817521997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCO2391055410,Mazda Cx-9 2.5 Grand Touring Signature 2022,used,https://carro.mercadolibre.com.co/MCO-23910554...,MCO1744,MCO21073776,MCO-CARS_AND_VANS,COP,180000000,1819774134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCO1426712899,Ram V700 1.4 Rapid 2022,used,https://carro.mercadolibre.com.co/MCO-14267128...,MCO1744,MCO35689514,MCO-CARS_AND_VANS,COP,45000000,1817495371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
